In [20]:
!pip install geopy
import pandas as pd
import datetime
import time
from geopy.distance import geodesic

In [2]:
df = pd.read_csv("201809-citibike-tripdata.csv")

In [3]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1635,2018-09-01 00:00:05.2690,2018-09-01 00:27:20.6340,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,132,2018-09-01 00:00:11.2810,2018-09-01 00:02:23.4810,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0
2,3337,2018-09-01 00:00:20.6490,2018-09-01 00:55:58.5470,3142.0,1 Ave & E 62 St,40.761227,-73.960940,3384.0,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1975,1
3,436,2018-09-01 00:00:21.7460,2018-09-01 00:07:38.5830,308.0,St James Pl & Oliver St,40.713079,-73.998512,3690.0,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,1984,2
4,8457,2018-09-01 00:00:27.3150,2018-09-01 02:21:25.3080,345.0,W 13 St & 6 Ave,40.736494,-73.997044,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1994,1


In [4]:
df.shape # Общее количество строк и столбцов в датасете

(1877884, 15)

In [5]:
round(df["tripduration"].mean() / 60, 2) # Средняя длина поездок в минутах

16.13

In [6]:
len(df[df["end station id"] == df["start station id"]])
# Колчичество поездок, которые начались и закончились на одной станции

41364

In [7]:
df.groupby(["bikeid"])["bikeid"].count().nlargest(1) # Самый используемый байк по количеству поездок

bikeid
33875    559
Name: bikeid, dtype: int64

In [8]:
df.groupby(["bikeid"]).agg({"tripduration": "mean"}).nlargest(5, columns = ["tripduration"])
# Байк с самой выскокой средней продолжительностью поездок

,tripduration
bikeid,
17548,488022.400000
28520,417717.000000
17431,394605.833333
19083,335620.142857
18041,103932.100000


In [9]:
len(df[df["start station id"].isnull()]) # Количество строк, в которых отсутствует start_station_id

716

In [10]:
df.groupby(['usertype']).agg({'tripduration': "mean"}).apply(lambda x: round(x['tripduration'] / 60, 2), axis = 1)
# Средняя продолжительность поездки в минутах в зависимости от типа подписки

usertype
Customer      33.42
Subscriber    13.33
dtype: float64

In [14]:
round(df[df["end station id"] != df["start station id"]].apply(
    lambda row : geodesic(
        (row["start station latitude"], row["start station longitude"]),
        (row["end station latitude"], row["end station longitude"])                          
    ).kilometers
    ,axis = 1).mean(), 2) # Средняя длинна поездок в километрах с точностью до 2 знака

1.85

In [21]:
df['end_hour'] = df['stoptime'].apply(lambda x: datetime.datetime.fromtimestamp(
            time.mktime(datetime.datetime.strptime(x.strip(), "%Y-%m-%d %H:%M:%S.%f").timetuple())).hour)

In [22]:
df['start_hour'] = df['starttime'].apply(lambda x: datetime.datetime.fromtimestamp(
                                        time.mktime(datetime.datetime.strptime(x.strip(), "%Y-%m-%d %H:%M:%S.%f").timetuple())).hour)

In [23]:
df[(df.start_hour.isin([18, 19, 20]))]['start station id'].value_counts().head()
# Выберите станцию (start station id) с максимальным количеством отправлений с 18 до 20 вечера

519.0    3181
426.0    2701
402.0    2680
459.0    2605
477.0    2418
Name: start station id, dtype: int64

In [25]:
end_stations = [3140, 3106, 3116, 369]
df[(df.end_hour.isin([6,7,8,9,10])) &
         (df['end station id'].isin(end_stations))]['end station id'].unique()
# Выберите станции(end station id), в которые приезжают с 6 до 10 утра

array([3106., 3116., 3140.,  369.])